# Tutorial: Controlled Stream Generation

We will generate words and a Lexicon with minimal feature overlap between the words. Next, we introduce the 3 main ways to generate random streams, depending on how the transition probabilities (TPs) of their syllables are structured: word-structured TPs, fully random (uniform) TPs, and position-controlled TPs.

First, we generate/reload the words register (see arc types tutorial).

In [ ]:
import os

from arc import load_words, load_phonemes, make_syllables, make_words

FORCE_RECOMPUTE = False

if os.path.exists("words.json") and not FORCE_RECOMPUTE:
    print("Load words...")
    words = load_words("words.json")
    print(words)
else:
    print("Load phonemes...")
    phonemes = load_phonemes()
    print(phonemes)

    print("Make syllables...")
    syllables = make_syllables(phonemes)
    print(syllables)

    print("Make words...")
    words = make_words(syllables)
    print(words)

    print("Save words ...")
    words.save("words.json")

## Lexicon

Now we generate lexica with minimal feature repetitiveness. Let's start with 4 words each. 

By default, the function will generate 5 `Lexicon`s max. Let's generate 2 and print some info.

In [ ]:
from arc import make_lexicons

lexicons = make_lexicons(words, n_lexicons=20, n_words=4)

for lexicon in lexicons:
    print("Lexicon:", lexicon)
    print("Info:", lexicon.info)
    print("")

By default, Lexicons with the minimum possible cumulative overlap between the word features will be generated first, starting at zero overlap. If it is not possible to generate all the requested Lexicons with the given parameters, the allowed overlap will be increased, which will be indicated by a warning message.

This process will be repeated, until any of the following statements is true
- the requested number of Lexicons has been generated
- the maximum allowed overlap is reached (set via `max_overlap`)
- the set of all word combinations is exhausted

If one or more Lexicons is returned, their info fields hold the cumulative overlap between all word pairs that is achieved by the Lexicon as well as the maximum pairwise overlap used.

## Compatible Streams

We want to generate a complete set of compatible lexicons for our study, i.e. to generate a compatible set of streams for testing statistical learning hypotheses. If `streams` is empty, try increasing the allowed maximum rythmicity).

As you can see, the `.info` field holds some useful information about the generated stream, i.e. which Lexicon has been used to generate it, the rythmicity indexes achieved for each feature, and which randomization/TP-structure mode has been used.

In [ ]:
import yaml
from arc import Register
import numpy as np

def print_stream(stream):
    stream_dict = {}
    stream_dict["stream"] = stream.id
    stream_dict["stream_info"] = {}
    for k, v in stream.info.items():
        if type(v) is Register:
            v = str(v)
        if type(v) is np.int64:
            v = int(v)
        if type(v) is tuple:
            v = list(v)
        stream_dict["stream_info"][k] = v
    yaml_str = yaml.dump(stream_dict, allow_unicode=True)
    print(yaml_str)

def print_streams_summary(streams):
    streams_summary = {}
    streams_summary["streams_info"] = streams.info
    streams_summary["streams_info"]["lexicons"] = list(set([str(stream.info["lexicon"]) for stream in streams]))
    streams_summary["streams_info"]["tp_modes"] = list(streams_summary["streams_info"]["tp_modes"])
    yaml_str = yaml.dump(streams_summary, allow_unicode=True)
    print(yaml_str)
    print("")
    for stream in streams:
        print_stream(stream)
        print("")

In [ ]:
from arc import make_streams
streams = make_streams(lexicons, max_rhythmicity=0.1)

print_streams_summary(streams)

# Compare Lexicons

Streams based on controlled lexicons (ours)

In [ ]:
N_LEXICONS = 10
N_REPS = 5

In [ ]:
from arc import make_streams

controlled_streams = make_streams(
    lexicons, 
    max_rhythmicity=None, 
    num_repetitions=N_REPS,
    n_lexicon_streams=N_LEXICONS
)

print_streams_summary(controlled_streams)

len(controlled_streams)

... vs. streams based on random / uncontrolled lexicons (baseline).

In [ ]:
random_lexicons = make_lexicons(words, n_lexicons=20, n_words=4, control_features=False)

random_streams = make_streams(
    lexicons, 
    max_rhythmicity=None, 
    num_repetitions=N_REPS,
    n_lexicon_streams=N_LEXICONS
)

print_streams_summary(random_streams)

In [ ]:
import pandas as pd

data = {"control": [], "lexicon": [], "feature": [], "PRI": [], "stream_tp_mode": []}
for control, streams in [("ours", controlled_streams), ("baseline", random_streams)]:
    for stream in streams:
        for k, v in stream.info["rhythmicity_indexes"].items():
            data["feature"].append(k)
            data["PRI"].append(v)
            data["control"].append(control)
            data["lexicon"].append(str(stream.info["lexicon"]))
            data["stream_tp_mode"].append(stream.info["stream_tp_mode"])

df = pd.DataFrame(data)

df

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

_, axs = plt.subplots(3, 1, figsize=(25, 25))

for i, tp_mode in enumerate(controlled_streams.info["tp_modes"]):
    sns.boxplot(df[df["stream_tp_mode"] == tp_mode], x="feature", y="PRI", hue="control", ax=axs[i])
    axs[i].set_title(f"Stream TP mode: {tp_mode}")

plt.savefig("lexicon_pri.pdf")